In [ ]:
# Import necessary libraries
import os
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
from collections import Counter

In [ ]:
# Function for preprocessing MP3 files
def load_mp3_16k_mono(filename):
    res = tfio.audio.AudioIOTensor(filename)
    tensor = res.to_tensor()
    tensor = tf.math.reduce_sum(tensor, axis=1) / 2
    sample_rate = res.rate
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(tensor, rate_in=sample_rate, rate_out=16000)
    return wav

# Function for converting clips into windowed spectrograms
def preprocess_mp3(sample, index):
    sample = sample[0]
    zero_padding = tf.zeros([48000] - tf.shape(sample), dtype=tf.float32)
    wav = tf.concat([zero_padding, sample],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [ ]:
# Load the model
model_path = (r'C:\Users\bianc\Desktop\Model')
model = tf.keras.models.load_model(model_path)

In [ ]:
results = {}
# Iterate through the audio files in the specified directory
for file in os.listdir(os.path.join('data', 'testfiles')):
    FILEPATH = os.path.join('data','testfiles', file)
    # Load the MP3 file and process it
    wav = load_mp3_16k_mono(FILEPATH)
    audio_slices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=48000, sequence_stride=48000, batch_size=1)
    audio_slices = audio_slices.map(preprocess_mp3)
    audio_slices = audio_slices.batch(64)
    # Make predictions for the audio slices
    y_predicted_probabilities = model.predict(audio_slices)
    # Store the predicted probabilities for this file
    results[file] = y_predicted_probabilities


class_preds = {}
# Iterate through the results and determine the class predictions
for file, y_predicted_probabilities in results.items():
    class_preds[file] = [np.argmax(probabilities) for probabilities in y_predicted_probabilities]


postprocessed = {}
for file, scores in class_preds.items():
    data = Counter(scores)
    postprocessed[file] = data.most_common(1)[0][0]
postprocessed